### DESCRIPTION
Script to compute demographic and morphological traits such as biovolume (µm3), surface area (µm2), surface/volume ratio, density (cells/L), cell carbon content (pg), total biovolume (µm³/L), total carbon content (pg/L). 
### INPUT VARIABLES
- datain: the input file
- CalcType: the computation type simplified or advanced
- CompTraits: one or more among biovolume,totalbiovolume,density,surfacearea,surfacevolumeratio,cellcarboncontent,totalcarboncontent
- CountingStrategy: one among density0,density1,density2,density3 

### OUTPUT 
file in .csv format, containing the original input data and the new calculated traits

In [1]:
# input parameters
# install.packages("dplyr",repos = "http://cran.us.r-project.org")

# dplyr = ''
# library(dplyr)

# param_hostname = readline(prompt = "Enter your hostname: ")
# param_login = readline(prompt = "Enter your username: ")
# param_password = readline(prompt = "Enter your password: ")

param_hostname = ''
param_login = ''
param_password = ''

conf_datain1 = "Phytoplankton__Progetto_Strategico_2009_2012_Australia.csv"
conf_datain2 = '2_FILEinformativo_OPERATORE.csv'

conf_output = '/tmp/data/'

param_CalcType = 'advanced'

conf_biovolume = 1 # if 1 it is calculated, if 0 it is not calculated
conf_totalbiovolume = 1
conf_density = 1
conf_surfacearea = 1
conf_surfacevolumeratio = 0
conf_cellcarboncontent = 1
conf_totalcarboncontent = 1

In [6]:
# S3 read input cvs

#Download file 
install.packages("RCurl",repos = "http://cran.us.r-project.org")
RCurl = ''
library(RCurl)
install.packages("httr",repos = "http://cran.us.r-project.org")
httr = ''
library(httr)

auth = basicTextGatherer()
cred = paste(param_login, param_password, sep = ":")

file_name = "Phytoplankton__Progetto_Strategico_2009_2012_Australia.csv"
dest_1 = paste(conf_output, file_name,sep = "")
download_file = paste0(param_hostname,file_name)
print(download_file)
file_content <- getURL(download_file, curl = getCurlHandle(userpwd = cred))
write(file_content, file = dest_1)


file_name = "2_FILEinformativo_OPERATORE.csv"
dest_2 = paste(conf_output, file_name,sep = "")
download_file = paste0(param_hostname,file_name)
print(download_file)
file_content <- getURL(download_file, curl = getCurlHandle(userpwd = cred))
write(file_content, file = dest_2)


countingStrategy = ''
if (conf_density==1) {countingStrategy <- 'density0'}

index = 0

# read input cvs
df.datain = read.csv(dest_1,stringsAsFactors=FALSE,sep = ";", dec = ".")
# df.datain=read.csv(text=file_content,stringsAsFactors=FALSE,sep = ";", dec = ".")
head(df.datain, n = 3)
df.datain[,'measurementremarks'] = tolower(df.datain[,'measurementremarks']) # eliminate capital letters
df.datain[,'index'] = c(1:nrow(df.datain)) # needed to restore rows order later

# read support cvs file
df.operator=read.csv(dest_2,stringsAsFactors=FALSE,sep = ",", dec = ".") # load internal database 
head(df.operator, n = 3)
df.operator[df.operator==('no')]<-NA
df.operator[df.operator==('see note')]<-NA

# merge dataframes
df.merged = merge(x = df.datain, y = df.operator, by = c("scientificname","measurementremarks"), all.x = TRUE)
head(df.merged, n = 3)

# check if mandatory fields are present
if(!'diameterofsedimentationchamber'%in%names(df.merged))df.merged[,'diameterofsedimentationchamber']=NA
if(!'diameteroffieldofview'%in%names(df.merged))df.merged[,'diameteroffieldofview']=NA
if(!'transectcounting'%in%names(df.merged))df.merged[,'transectcounting']=NA
if(!'numberofcountedfields'%in%names(df.merged))df.merged[,'numberofcountedfields']=df.merged[,'transectcounting']
if(!'numberoftransects'%in%names(df.merged))df.merged[,'numberoftransects']=df.merged[,'transectcounting']
if(!'settlingvolume'%in%names(df.merged))df.merged[,'settlingvolume']=NA
if(!'dilutionfactor'%in%names(df.merged))df.merged[,'dilutionfactor']=1

# save merged dataframe as csv
output_dfmerged_1 = paste(conf_output, "dfmerged_1.csv",sep = "")
write.table(df.merged,output_dfmerged_1,row.names=FALSE,sep = ";",dec = ".",quote=FALSE)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



[1] "https://lifewatch.lab.uvalight.net/webdav/vl-phytoplankton/mariantonietta/Phytoplankton__Progetto_Strategico_2009_2012_Australia.csv"
[1] "https://lifewatch.lab.uvalight.net/webdav/vl-phytoplankton/mariantonietta/2_FILEinformativo_OPERATORE.csv"


,catalognumber,organismquantity,organismquantitytype,eventid,parenteventid,year,month,day,country,countrycode,⋯,d11,d22,d33,d44,volumeeqsphere,volumeeqcylinder,classcode,volumeofsedimentationchamber,transectcounting,eunishabitatstypename
,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<chr>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>,<int>,<int>,<int>,<chr>
1,10681,1,individual,a,10,2011,3,4,australia,au,⋯,NA,NA,NA,NA,1413.12,900.773,627,5,28,saline coastal lagoon
2,13937,1,individual,c,12,2011,3,4,australia,au,⋯,NA,NA,NA,NA,43649.70,28287.800,409,5,19,saline coastal lagoon
3,14916,1,individual,b,13,2011,3,4,australia,au,⋯,NA,NA,NA,NA,109654.00,53036.400,409,5,22,littoral sediment


,classcode,scientificname,shape,missingdimension,formulaformissingdimension,hd,formulaforbiovolume,formulaforsurface,formulaforweight1,formulaforweight2,missingdimensionsimplified,formulaformissingdimensionsimplified,formulaforbiovolumesimplified,formulaforsurfacesimplified,measurementremarks,mostlk
,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,Ceratoneis closterium,prolate spheroid+2 cylinder,no,no,NA,((pi/6)*(d^2)*h)+((pi/4)*(d1^2)*h1)+((pi/4)*(d1^2)*h2),((pi/2)*d)*(d+(((h^2)/sqrt((h^2)-(d^2))))*(asin((sqrt((h^2)-(d^2)))/h)))+((pi*d1)*(h1+h2)),0.26*Biovolume^0.86,0.288*Biovolume^0.811,no,no,(pi/4)*(width^2)*length,pi*width*((width/2)+length),unique,y
2,5,Dinophyceae,sphere,no,no,NA,(pi/6)*d^3,(pi*(d^2)),0.26*Biovolume^0.86,0.216*Biovolume^0.939,no,no,(pi/6)*length^3,(pi*(length^2)),morethan20,y
3,6,Navicula,prism on elliptic base,c,hd*a,0.2725399,(pi/4)*a*b*c,(pi/2)*((a*b)+(c*(a+b))),0.26*Biovolume^0.86,0.288*Biovolume^0.811,c,hd*length,(pi/4)*length*width*c,(pi/2)*((length*width)+(c*(length+length))),valvar,y


,scientificname,measurementremarks,catalognumber,organismquantity,organismquantitytype,eventid,parenteventid,year,month,day,⋯,hd,formulaforbiovolume,formulaforsurface,formulaforweight1,formulaforweight2,missingdimensionsimplified,formulaformissingdimensionsimplified,formulaforbiovolumesimplified,formulaforsurfacesimplified,mostlk
,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,⋯,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Achnanthaceae,girdle,10681,1,individual,a,10,2011,3,4,⋯,0.3416667,(pi/4)*a*b*c,(pi/2)*((a*b)+(c*(a+b))),0.26*Biovolume^0.86,0.288*Biovolume^0.811,b,hd*length,(pi/4)*length*b*width,(pi/2)*((length*b)+(width*(length+b))),n
2,Akashiwo sanguinea,frontal,13937,1,individual,c,12,2011,3,4,⋯,0.4072569,(pi/6)*b*c*h,(pi/4)*(b+c)*(((b+c)/2)+(((2*(h^2))/((sqrt(4*h^2-((b+c)^2)))))*asin((sqrt(4*h^2-((b+c)^2)))/(2*h)))),0.26*Biovolume^0.86,0.216*Biovolume^0.939,c,hd*length,(pi/6)*width*c*length,(pi/4)*(width+c)*(((width+c)/2)+(((2*(length^2))/((sqrt(4*length^2-((width+c)^2)))))*asin((sqrt(4*length^2-((width+c)^2)))/(2*length)))),y
3,Akashiwo sanguinea,frontal,14916,1,individual,b,13,2011,3,4,⋯,0.4072569,(pi/6)*b*c*h,(pi/4)*(b+c)*(((b+c)/2)+(((2*(h^2))/((sqrt(4*h^2-((b+c)^2)))))*asin((sqrt(4*h^2-((b+c)^2)))/(2*h)))),0.26*Biovolume^0.86,0.216*Biovolume^0.939,c,hd*length,(pi/6)*width*c*length,(pi/4)*(width+c)*(((width+c)/2)+(((2*(length^2))/((sqrt(4*length^2-((width+c)^2)))))*asin((sqrt(4*length^2-((width+c)^2)))/(2*length)))),y


In [7]:
# S4 missing_dimensions

df.merged = read.csv(output_dfmerged_1,stringsAsFactors=FALSE,sep = ";", dec = ".")

# we need to initialize these variables so they are not showing up as inputs
formulaformissingdimension = '' 
formulaformissingdimensionsimplified = ''
df.temp = ''  
md.form = ''
df.temp2 = ''
index = ''
md = ''
df.merged.concat = '' 
md.formulas = ''

if(param_CalcType=='advanced'){
  df.merged.concat = df.merged[is.na(df.merged[,'formulaformissingdimension']),]
  md.formulas = unique(df.merged[!is.na(df.merged[,'formulaformissingdimension']),'formulaformissingdimension'])
  for(md.form in md.formulas){
    df.temp = subset(df.merged,formulaformissingdimension==md.form)
    for(md in unique(df.temp[,'missingdimension'])){
      df.temp2 = df.temp[df.temp[,'missingdimension']==md,]
      df.temp2[[md]] = round(with(df.temp2,eval(parse(text=md.form))),2)
      df.merged.concat = rbind(df.merged.concat,df.temp2)
    }
  }
  df.merged.concat = df.merged.concat[order(df.merged.concat[,'index']),]
  df.merged = df.merged.concat
} else if(param_CalcType=='simplified'){
  df.merged.concat = df.merged[is.na(df.merged[,'formulaformissingdimensionsimplified']),]
  md.formulas = unique(df.merged[!is.na(df.merged[,'formulaformissingdimensionsimplified']),'formulaformissingdimensionsimplified'])
  for(md.form in md.formulas){
    df.temp = subset(df.merged,formulaformissingdimensionsimplified==md.form)
    for(md in unique(df.temp[,'missingdimensionsimplified'])){
      df.temp2 = df.temp[df.temp[,'missingdimensionsimplified']==md,]
      df.temp2[[md]] = round(with(df.temp2,eval(parse(text=md.form))),2)
      df.merged.concat = rbind(df.merged.concat,df.temp2)
    }
  }
  df.merged.concat = df.merged.concat[order(df.merged.concat[,'index']),]
  df.merged = df.merged.concat
}

# save merged dataframe as csv
output_dfmerged_2 = paste(conf_output, "dfmerged_2.csv",sep = "")
write.table(df.merged,output_dfmerged_2,row.names=FALSE,sep = ";",dec = ".",quote=FALSE)


In [8]:
# S5 biovolume

df.merged=read.csv(output_dfmerged_2,stringsAsFactors=FALSE,sep = ";", dec = ".")

# we need to initialize these variables so they are not showing up as inputs
formulaforbiovolume = '' 
formulaforbiovolumesimplified = '' 
df.temp = ''  
bv.form = ''
df.merged.concat = '' 
bv.formulas = ''

if(conf_biovolume==1){
  if(param_CalcType=='advanced'){
    df.merged[,'biovolume'] = rep(NA,length=nrow(df.merged))
    df.merged.concat = df.merged[is.na(df.merged[,'formulaforbiovolume']),]
    bv.formulas = unique(df.merged[!is.na(df.merged[,'formulaforbiovolume']),'formulaforbiovolume'])
    for(bv.form in bv.formulas){
      df.temp = subset(df.merged,formulaforbiovolume==bv.form)
      df.temp[,'biovolume'] = round(with(df.temp,eval(parse(text=bv.form))),2)
      df.merged.concat = rbind(df.merged.concat,df.temp)
    }
    df.merged.concat = df.merged.concat[order(df.merged.concat[,'index']),]
    df.merged = df.merged.concat
  }
  else if(param_CalcType=='simplified'){
    df.merged[,'biovolume'] = rep(NA,length=nrow(df.merged))
    df.merged.concat = df.merged[is.na(df.merged[,'formulaforbiovolumesimplified']),]
    bv.formulas = unique(df.merged[!is.na(df.merged[,'formulaforbiovolumesimplified']),'formulaforbiovolumesimplified'])
    for(bv.form in bv.formulas){
      df.temp = subset(df.merged,formulaforbiovolumesimplified==bv.form)
      df.temp[,'biovolume'] = round(with(df.temp,eval(parse(text=bv.form))),2)
      df.merged.concat <- rbind(df.merged.concat,df.temp)
    }
    df.merged.concat = df.merged.concat[order(df.merged.concat[,'index']),]
    df.merged = df.merged.concat
  }
} 

# save merged dataframe as csv
output_dfmerged_3 = paste(conf_output, "dfmerged_3.csv",sep = "")
write.table(df.merged,output_dfmerged_3,row.names=FALSE,sep = ";",dec = ".",quote=FALSE)

In [9]:
# S6 cellcarboncontent

df.merged=read.csv(output_dfmerged_3,stringsAsFactors=FALSE,sep = ";", dec = ".")

# we need to initialize these variables so they are not showing up as inputs
formulaforweight1 = '' 
formulaforweight2 = '' 
biovolume = ''
cellcarboncontent = ''
df.temp = ''  
cc.form = ''
df.merged.concat = '' 
cc.formulas1 = ''
cc.formulas2 = ''
df.cc = ''
df.cc1 = ''
df.cc2 = ''


if(conf_cellcarboncontent==1){
  df.merged[,'cellcarboncontent'] = rep(NA,length=nrow(df.merged))
  if(conf_biovolume==1){
    df.merged.concat = df.merged[is.na(df.merged[,'biovolume']),]
    df.cc = df.merged[!is.na(df.merged[,'biovolume']),]
    df.cc1 = subset(df.cc,biovolume <= 3000)
    df.cc2 = subset(df.cc,biovolume > 3000)
    cc.formulas1 = unique(df.merged[!is.na(df.merged[,'formulaforweight1']),'formulaforweight1'])
    for(cc.form in cc.formulas1){
      df.temp = subset(df.cc1,formulaforweight1==cc.form)
      df.temp[,'cellcarboncontent'] = round(with(df.temp,eval(parse(text=tolower(cc.form)))),2)
      df.merged.concat = rbind(df.merged.concat,df.temp)
    }
    cc.formulas2 = unique(df.merged[!is.na(df.merged[,'formulaforweight2']),'formulaforweight2'])
    for(cc.form in cc.formulas2){
      df.temp = subset(df.cc2,formulaforweight2==cc.form)
      df.temp$cellcarboncontent = round(with(df.temp,eval(parse(text=tolower(cc.form)))),2)
      df.merged.concat = rbind(df.merged.concat,df.temp)
    }
    df.merged.concat = df.merged.concat[order(df.merged.concat[,'index']),]
    df.merged = df.merged.concat
  }
}

# save merged dataframe as csv
output_dfmerged_4 = paste(conf_output, "dfmerged_4.csv",sep = "")
write.table(df.merged,output_dfmerged_4,row.names=FALSE,sep = ";",dec = ".",quote=FALSE)




In [10]:
# S7 other_traits

df.merged=read.csv(output_dfmerged_4,stringsAsFactors=FALSE,sep = ";", dec = ".")

if (conf_density==1) {countingStrategy <- 'density0'}

# we need to initialize these variables so they are not showing up as inputs
volumeofsedimentationchamber = '' 
df.temp = ''
df.temp1 = ''
df.temp2 = ''  
df.temp3 = ''  
df.temp4 = ''  
df.temp5 = ''  
df.merged.concat = '' 
pi = 3.14159


# density

if(conf_density==1){
  df.merged[,'density'] = rep(NA,length=nrow(df.merged))
  # default method to calculate the density
  if(countingStrategy=='density0'){  
    df.merged.concat = df.merged[(is.na(df.merged[,'volumeofsedimentationchamber'])) & (is.na(df.merged[,'transectcounting'])),]
    df.temp = df.merged[!is.na(df.merged[,'volumeofsedimentationchamber']) & !is.na(df.merged[,'transectcounting']),]
    df.temp1 = subset(df.temp,volumeofsedimentationchamber <= 5)
    df.temp1[,'density'] = df.temp1[,'organismquantity']/df.temp1[,'transectcounting']*1000/0.001979
    df.merged.concat = rbind(df.merged.concat,df.temp1)
    df.temp2 = subset(df.temp,(volumeofsedimentationchamber > 5) & (volumeofsedimentationchamber <= 10))
    df.temp2[,'density'] = df.temp2[,'organismquantity']/df.temp2[,'transectcounting']*1000/0.00365
    df.merged.concat = rbind(df.merged.concat,df.temp2)
    df.temp3 = subset(df.temp,(volumeofsedimentationchamber > 10) & (volumeofsedimentationchamber <= 25))
    df.temp3[,'density'] = df.temp3[,'organismquantity']/df.temp3[,'transectcounting']*1000/0.010555
    df.merged.concat = rbind(df.merged.concat,df.temp3)
    df.temp4 = subset(df.temp,(volumeofsedimentationchamber > 25) & (volumeofsedimentationchamber <= 50))
    df.temp4[,'density'] = df.temp4[,'organismquantity']/df.temp4[,'transectcounting']*1000/0.021703
    df.merged.concat = rbind(df.merged.concat,df.temp4)
    df.temp5 = subset(df.temp,volumeofsedimentationchamber > 50)
    df.temp5[,'density'] = df.temp5[,'organismquantity']/df.temp5[,'transectcounting']*1000/0.041598
    df.merged.concat = rbind(df.merged.concat,df.temp5)
    df.merged.concat = df.merged.concat[order(df.merged.concat[,'index']),]
    df.merged = df.merged.concat
    df.merged[,'density'] = round(df.merged[,'density'],2)
  }
  # counts per random field
  else if(countingStrategy=='density1'){
    df.merged[,'areaofsedimentationchamber'] = ((df.merged[,'diameterofsedimentationchamber']/2)^2)*pi
    df.merged[,'areaofcountingfield'] = ((df.merged[,'diameteroffieldofview']/2)^2)*pi
    df.merged[,'density'] = round(df.merged[,'organismquantity']*1000*df.merged[,'areaofsedimentationchamber']/df.merged[,'numberofcountedfields']*df.merged[,'areaofcountingfield']*df.merged[,'settlingvolume'],2)
  }
  # counts per diameter transects
  else if(countingStrategy=='density2'){
    df.merged[,'density'] = round(((df.merged[,'organismquantity']/df.merged[,'numberoftransects'])*(pi/4)*(df.merged[,'diameterofsedimentationchamber']/df.merged[,'diameteroffieldofview']))*1000/df.merged[,'settlingvolume'],2)
  }
  # counting method for whole chamber
  else if(countingStrategy=='density3'){
    df.merged[,'density'] = round((df.merged[,'organismquantity']*1000)/df.merged[,'settlingvolume'],2)
  }
  df.merged[,'density'] = df.merged[,'density']/df.merged[,'dilutionfactor']
}   
    

# totalbiovolume

if(conf_totalbiovolume==1){
  if((conf_density==0) & (!'density'%in%names(df.merged))) df.merged[,'density']=NA
  if((conf_biovolume==0) & (!'biovolume'%in%names(df.merged))) df.merged[,'biovolume']=NA
  df.merged[,'totalbiovolume'] = round(df.merged[,'density']*df.merged[,'biovolume'],2)
}
    
    
# surfacevolumeratio
if(conf_surfacevolumeratio==1){
  if((conf_surfacearea==0) & (!'surfacearea'%in%names(df.merged))) df.merged[,'surfacearea']=NA
  if((conf_biovolume==0) & (!'biovolume'%in%names(df.merged))) df.merged[,'biovolume']=NA
  df.merged[,'surfacevolumeratio']=round(df.merged[,'surfacearea']/df.merged[,'biovolume'],2)
}
    
    
# totalcarboncontent

if(conf_totalcarboncontent==1){
  if((conf_density==0) & (!'density'%in%names(df.merged))) df.merged[,'density']=NA
  if((conf_cellcarboncontent==0) & (!'cellcarboncontent'%in%names(df.merged))) df.merged[,'cellcarboncontent']=NA
  df.merged[,'totalcarboncontent']=round(df.merged[,'density']*df.merged[,'cellcarboncontent'],2)
}
   
# save merged dataframe as csv
output_dfmerged_5 = paste(conf_output, "dfmerged_5.csv",sep = "")
write.table(df.merged,output_dfmerged_5,row.names=FALSE,sep = ";",dec = ".",quote=FALSE)

In [11]:
#S8 TraitTotalbiovolume

biovolume = '' 
density = ''
df.merged=read.csv(output_dfmerged_5,stringsAsFactors=FALSE,sep = ";", dec = ".")

if(conf_totalbiovolume==1){
  if((conf_density==0) & (!'density'%in%names(df.merged))) df.merged$density<-NA
  if((conf_biovolume==0) & (!'biovolume'%in%names(df.merged))) df.merged$biovolume<-NA
  TBV_calc = round(df.merged$density*df.merged$biovolume,2)
}

# save merged dataframe as csv
output_dfmerged_6 = paste(conf_output, "dfmerged_6.csv",sep = "")
write.table(df.merged,output_dfmerged_6,row.names=FALSE,sep = ";",dec = ".",quote=FALSE)


In [12]:
# S9 TraitSurfacearea

index = 0

df.merged=read.csv(output_dfmerged_6,stringsAsFactors=FALSE,sep = ";", dec = ".")
surfacearea = ''
formulaforsurfacesimplified = '' 
formulaforsurface = ''

CalcType=param_CalcType

if(conf_surfacearea==1){
  if(CalcType=='advanced'){
    df.merged$surfacearea <- rep(NA,length=nrow(df.merged))
    df.merged.concat <- df.merged[is.na(df.merged$formulaforsurface),]
    sa.formulas <- unique(df.merged[!is.na(df.merged$formulaforsurface),]$formulaforsurface)
    for(sa.form in sa.formulas){
      df.temp <- subset(df.merged,formulaforsurface==sa.form)
      df.temp$surfacearea <- round(with(df.temp,eval(parse(text=sa.form))),2)
      df.merged.concat <- rbind(df.merged.concat,df.temp)
    }
    df.merged.concat <- df.merged.concat[order(df.merged.concat$index),]
    df.merged <- df.merged.concat
    SA_calc <- df.merged$surfacearea
  }
  else if(CalcType=='simplified'){
    df.merged$surfacearea <- rep(NA,length=nrow(df.merged))
    df.merged.concat <- df.merged[is.na(df.merged$formulaforsurfacesimplified),]
    sa.formulas <- unique(df.merged[!is.na(df.merged$formulaforsurfacesimplified),]$formulaforsurfacesimplified)
    for(sa.form in sa.formulas){
      df.temp <- subset(df.merged,formulaforsurfacesimplified==sa.form)
      df.temp$surfacearea <- round(with(df.temp,eval(parse(text=sa.form))),2)
      df.merged.concat <- rbind(df.merged.concat,df.temp)
    }
    df.merged.concat <- df.merged.concat[order(df.merged.concat$index),]
    df.merged <- df.merged.concat
    SA_calc <- df.merged$surfacearea
  }
}

# save merged dataframe as csv
output_dfmerged_7 = paste(conf_output, "dfmerged_7.csv",sep = "")
write.table(df.merged,output_dfmerged_7,row.names=FALSE,sep = ";",dec = ".",quote=FALSE)

Warning message in sqrt((h^2) - (d^2)):
“NaNs produced”
Warning message in sqrt((h^2) - (d^2)):
“NaNs produced”
Warning message in sqrt(4 * h^2 - ((b + c)^2)):
“NaNs produced”
Warning message in sqrt(4 * h^2 - ((b + c)^2)):
“NaNs produced”


In [13]:
# S10 TraitSurfacevolumeratio

biovolume = ''
surfacearea = ''
SVR_calc = 0.0

df.merged=read.csv(output_dfmerged_7,stringsAsFactors=FALSE,sep = ";", dec = ".")

if(conf_surfacevolumeratio==1){
  if((conf_surfacearea==0) & (!'surfacearea'%in%names(df.merged))) df.merged$surfacearea<-NA
  if((conf_biovolume==0) & (!'biovolume'%in%names(df.merged))) df.merged$biovolume<-NA
  SVR_calc<-round(df.merged$surfacearea/df.merged$biovolume,2)
}


# save merged dataframe as csv
output_dfmerged_8 = paste(conf_output, "dfmerged_8.csv",sep = "")
write.table(df.merged,output_dfmerged_8,row.names=FALSE,sep = ";",dec = ".",quote=FALSE)

In [14]:
# S11 TraitTotalcarboncontent
df.merged=read.csv(output_dfmerged_8,stringsAsFactors=FALSE,sep = ";", dec = ".")

cellcarboncontent = ''
density = ''
TCC_calc = 0.0

if(conf_totalcarboncontent==1){
  if((conf_density==0) & (!'density'%in%names(df.merged))) df.merged$density<-NA
  if((conf_cellcarboncontent==0) & (!'cellcarboncontent'%in%names(df.merged))) df.merged$cellcarboncontent<-NA
  TCC_calc<-round(df.merged$density*df.merged$cellcarboncontent,2)
}

# save merged dataframe as csv
output_dfmerged_9 = paste(conf_output, "dfmerged_8.csv",sep = "")
write.table(df.merged,output_dfmerged_9,row.names=FALSE,sep = ";",dec = ".",quote=FALSE)


In [15]:
# S12 TraitBiovolume
index = 0


df.datain=read.csv(output_dfmerged_9,stringsAsFactors=FALSE,sep = ";", dec = ".")

BV=conf_biovolume 
TBV=conf_totalbiovolume
D=conf_density
SA=conf_surfacearea
SVR=conf_surfacevolumeratio
CC=conf_cellcarboncontent
TCC=conf_totalcarboncontent

D_calc = ''


cellcarboncontent = ''
biovolume = ''
surfacearea = ''
density = ''

if(BV==1) {
    BV_column=BV_calc
    if('biovolume'%in%names(df.datain)) df.datain<-subset(df.datain,select=-biovolume) # drop column if already present
    df.datain$biovolume <- BV_column # write column with the results at the end of the dataframe
    }
if(CC==1) {
    CC_column=CC_calc
    if('cellcarboncontent'%in%names(df.datain)) df.datain<-subset(df.datain,select=-cellcarboncontent)
    df.datain$cellcarboncontent <- CC_column
    }
if(D==1) {
    D_column=D_calc
    df.datain$density <- D_column
    }
if(TBV==1) {
    TBV_column=TBV_calc
    df.datain$totalbiovolume <- TBV_column
    }
if(SA==1) {
    SA_column=SA_calc
    if('surfacearea'%in%names(df.datain)) df.datain<-subset(df.datain,select=-surfacearea)
    df.datain$surfacearea <- SA_column
    }
if(SVR==1) {
    SVR_column=SVR_calc
    df.datain$surfacevolumeratio <- SVR_column
    }
if(TCC==1) {
    TCC_column=TCC_calc
    df.datain$totalcarboncontent <- TCC_column
    }

df.datain <- subset(df.datain,select = -index)

#write.table(df.datain,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/Output/TraitsOutput.csv',sep=''),row.names=FALSE,sep = ";",dec = ".",quote=FALSE) 


# save merged dataframe as csv
traits_output = paste(conf_output, "TraitsOutput.csv",sep = "")
write.table(df.datain,traits_output,row.names=FALSE,sep = ";",dec = ".",quote=FALSE)

file_content <- readBin(traits_output, what = "raw", n = file.info(traits_output)$size)
upload_file_name <- basename(traits_output)
# Create a PUT request
response <- httr::PUT(
  url = paste0(param_hostname, upload_file_name),
  body = file_content,
  httr::authenticate(user = param_login, password = param_password),
  verbose()
)
print(response)




ERROR: Error in eval(expr, envir, enclos): object 'BV_calc' not found
